In [1]:
from IPython.display import Image

In [2]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_01.png', width=500)

순차데이터 VS 시계열 데이터

시권스 모델링 종류

In [3]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_02.png', width=500)

RNN

In [4]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_03.png', width=500)

In [5]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_04.png', width=500)

RNN의 활성화 출력 계산

In [6]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_05.png', width=500)

In [7]:
import torch
import torch.nn as nn
torch.manual_seed(1)

In [8]:
rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)
#(배치크기, 시퀀스길이,특징벡터크기)

In [17]:
layer_name = ['입력에대한 가중치','은닉상태에 대한 가중치','바이어스','추가적인 바이어스(역방향 순환연결)']
for idx,weight in enumerate(rnn_layer.all_weights[0]):
  print(f"{layer_name[idx]}\ndata:{weight}\n\n")

입력에대한 가중치
data:Parameter containing:
tensor([[ 0.3643, -0.3121, -0.1371,  0.3319, -0.6657],
        [ 0.4241, -0.1455,  0.3597,  0.0983, -0.0866]], requires_grad=True)


은닉상태에 대한 가중치
data:Parameter containing:
tensor([[ 0.1961,  0.0349],
        [ 0.2583, -0.2756]], requires_grad=True)


바이어스
data:Parameter containing:
tensor([-0.0516, -0.0637], requires_grad=True)


추가적인 바이어스(역방향 순환연결)
data:Parameter containing:
tensor([ 0.1025, -0.0028], requires_grad=True)




In [20]:
w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0
print(f'w_xh size {w_xh.shape}')
print(f'w_hh size {w_hh.shape}')
print(f'b_xh size {b_xh.shape}')
print(f'b_hh size {b_hh.shape}')

w_xh size torch.Size([2, 5])
w_hh size torch.Size([2, 2])
b_xh size torch.Size([2])
b_hh size torch.Size([2])


In [21]:
# 각 시간 단계에서 길이가 5인 벡터로 이루어진 시퀀스
x_seq = torch.tensor([ [1.0]*5, [2.0]*5, [3.0]*5  ]  ).float()
x_seq

tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]])

In [23]:
# 간단한 RNN 출력
torch.reshape(x_seq, (1,3,5) ).shape  # 배치크기, 시퀀스 길이, 특징벡터의 크기

torch.Size([1, 3, 5])

In [29]:
output, hn = rnn_layer(torch.reshape(x_seq, (1,3,5) ))  # 배치크기, 시퀀스길이, 은닉상태의 크기

In [30]:
# 수동으로 출력 계산
out_man = []
for t in range(3):
  xt = torch.reshape(x_seq[t], (1,5))
  print(f'타임스텝{t}=>')
  print('   입력      :',xt.numpy())

  ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
  print('   은닉    :',ht.detach().numpy())

  if t>0:
    prev_h = out_man[t-1]
  else:
    prev_h = torch.zeros((ht.shape))
  ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
  ot = torch.tanh(ot)
  out_man.append(ot)
  print(' 출력(수동) : ', ot.detach().numpy())
  print(' RNN 출력 : ', output[:,t].detach().numpy() )
  print()




타임스텝0=>
   입력      : [[1. 1. 1. 1. 1.]]
   은닉    : [[-0.4701929   0.58639044]]
 출력(수동) :  [[-0.3519801   0.52525216]]
 RNN 출력 :  [[-0.3519801   0.52525216]]

타임스텝1=>
   입력      : [[2. 2. 2. 2. 2.]]
   은닉    : [[-0.88883156  1.2364398 ]]
 출력(수동) :  [[-0.68424344  0.76074266]]
 RNN 출력 :  [[-0.68424344  0.76074266]]

타임스텝2=>
   입력      : [[3. 3. 3. 3. 3.]]
   은닉    : [[-1.3074702  1.8864892]]
 출력(수동) :  [[-0.8649416  0.9046636]]
 RNN 출력 :  [[-0.8649416  0.9046636]]



긴 시퀀스 학습의 어려움

In [31]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_08.png', width=500)

LSTM 셀

In [32]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_09.png', width=500)